In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
root = "final.csv"
batch_size = 256
epochs = 200
learning_rate = 1e-3
seed = 42

In [3]:
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [4]:
class CsvDataset(Dataset):
    def __init__(self, path):
        super(CsvDataset).__init__()
        csv = pd.read_csv(path)
        csv_np = csv.to_numpy()
        for i in [1, 3, 4, 5, 6, 7, 8]:
            wordset = {word: idx for idx, word in enumerate(np.unique(csv_np[:,i]))}
            for row in range(len(csv_np)):
                csv_np[row][i] = wordset[csv_np[row][i]]
        self.ints = torch.from_numpy(np.array(csv_np[:,[1,3,4,5,6,7,8]], dtype="int"))
        self.floats = torch.from_numpy(np.array(csv_np[:,[2,9,10,11]], dtype="float")).float()
        self.target = torch.from_numpy(np.array(csv_np[:,[12]], dtype="float")).float()
        
        
    def __getitem__(self,idx):
        return self.ints[idx],self.floats[idx], self.target[idx]
    
    
    def __len__(self):
        return len(self.ints)

In [5]:
data_length = len(pd.read_csv(root))
train_length = int(data_length * 0.6)
test_length = int(data_length * 0.2)
val_length = data_length - train_length - test_length

train_dataset = CsvDataset(root)
train_dataset, test_dataset = random_split(train_dataset, [train_length, test_length+val_length])
test_dataset, val_dataset = random_split(test_dataset, [test_length, val_length])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = True)

In [6]:
class CsvNet(nn.Module):
    def __init__(self):
        super(CsvNet, self).__init__()
        self.emb1 = torch.nn.Embedding(8, 20)
        self.emb2 = torch.nn.Embedding(11, 20)
        self.emb3 = torch.nn.Embedding(20, 20)
        self.emb4 = torch.nn.Embedding(4, 20)
        self.emb5 = torch.nn.Embedding(4, 20)
        self.emb6 = torch.nn.Embedding(4, 20)
        self.emb7 = torch.nn.Embedding(7, 20)
        self.act = nn.ReLU()
        self.fc = nn.Linear(4, 80)
        self.fc1 = nn.Linear(220, 8192)
        self.fc2 = nn.Linear(8192, 8192)
        self.fc3 = nn.Linear(8192, 4096)
        self.fc4 = nn.Linear(4096, 2048)
        self.fc5 = nn.Linear(2048, 1)
        self.dropout = nn.Dropout()
    
    def forward(self, x, y):
        x1 = self.emb1(x[:,0])
        x2 = self.emb2(x[:,1])
        x3 = self.emb3(x[:,2])
        x4 = self.emb4(x[:,3])
        x5 = self.emb5(x[:,4])
        x6 = self.emb6(x[:,5])
        x7 = self.emb7(x[:,6])
        y = self.fc(y)
        x = torch.cat((x1, x2, x3, x4, x5, x6, x7, y), dim=1)
        x = self.dropout(self.act(self.fc1(x)))
        x = self.dropout(self.act(self.fc2(x)))
        x = self.dropout(self.act(self.fc3(x)))
        x = self.dropout(self.act(self.fc4(x)))
        return self.fc5(x)

In [7]:
model = CsvNet().to(device)

criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
for epoch in range(epochs):
    model.train()
    criterion.train()
    
    avg_loss = 0

    for X1, X2, Y in train_loader:
        X1 = X1.to(device)
        X2 = X2.to(device)
        Y = Y.to(device)

        model.zero_grad()  # why we use zero_grad?
        prediction = model(X1, X2)
        loss = torch.sqrt(criterion(prediction, Y)).to(device)
        loss.backward()
        optimizer.step()
        avg_loss += loss / len(train_loader)
    print(f'[Epoch: {epoch+1:>2}] Average loss: {avg_loss:.4f}, ', end='')
    
    model.eval()
    criterion.eval()
    with torch.no_grad():
        val_avg_loss = 0.
        for X1_val, X2_val, Y_val in val_loader:
            X1_val = X1_val.to(device)
            X2_val = X2_val.to(device)
            Y_val = Y_val.to(device)
            val_prediction = model(X1_val, X2_val)
            val_loss = torch.sqrt(criterion(val_prediction, Y_val)).to(device)
            val_avg_loss += val_loss / len(val_loader)
        
        print(f"val_loss: {val_avg_loss:.4f}")

[Epoch:  1] Average loss: 4664.2144, val_loss: 4720.9932
[Epoch:  2] Average loss: 4918.0645, val_loss: 4688.7866
[Epoch:  3] Average loss: 4134.2627, val_loss: 4060.3608
[Epoch:  4] Average loss: 3512.7324, val_loss: 3804.7524
[Epoch:  5] Average loss: 3319.3657, val_loss: 3227.0962
[Epoch:  6] Average loss: 3003.7322, val_loss: 3039.6377
[Epoch:  7] Average loss: 2735.9070, val_loss: 2687.7537
[Epoch:  8] Average loss: 2277.8770, val_loss: 1875.8375
[Epoch:  9] Average loss: 1803.1349, val_loss: 1865.1333
[Epoch: 10] Average loss: 1733.7134, val_loss: 1829.8857
[Epoch: 11] Average loss: 1662.5404, val_loss: 1816.6262
[Epoch: 12] Average loss: 1408.3654, val_loss: 2060.0247
[Epoch: 13] Average loss: 1241.3468, val_loss: 2015.5648
[Epoch: 14] Average loss: 1332.5330, val_loss: 1992.5648
[Epoch: 15] Average loss: 1694.7236, val_loss: 1712.1798
[Epoch: 16] Average loss: 1918.2107, val_loss: 2558.7036
[Epoch: 17] Average loss: 2299.2974, val_loss: 1782.4941
[Epoch: 18] Average loss: 1638.

[Epoch: 146] Average loss: 841.3782, val_loss: 1158.8446
[Epoch: 147] Average loss: 862.9371, val_loss: 1396.3071
[Epoch: 148] Average loss: 807.0493, val_loss: 1078.2390
[Epoch: 149] Average loss: 891.0037, val_loss: 860.9224
[Epoch: 150] Average loss: 748.2303, val_loss: 781.6959
[Epoch: 151] Average loss: 868.2858, val_loss: 1050.3909
[Epoch: 152] Average loss: 838.3925, val_loss: 1021.9572
[Epoch: 153] Average loss: 716.5667, val_loss: 1217.8055
[Epoch: 154] Average loss: 833.7059, val_loss: 681.2051
[Epoch: 155] Average loss: 879.2791, val_loss: 992.1744
[Epoch: 156] Average loss: 894.0251, val_loss: 790.3545
[Epoch: 157] Average loss: 909.4001, val_loss: 1041.2510
[Epoch: 158] Average loss: 709.2912, val_loss: 857.6290
[Epoch: 159] Average loss: 673.4729, val_loss: 915.7433
[Epoch: 160] Average loss: 832.3898, val_loss: 844.0375
[Epoch: 161] Average loss: 780.5184, val_loss: 794.4377
[Epoch: 162] Average loss: 881.8588, val_loss: 1094.9105
[Epoch: 163] Average loss: 1429.8187, va

In [9]:
model.eval()
criterion.eval()
ss_tot = 0
ss_res = 0
with torch.no_grad():
    for X1_test, X2_test, Y_test in test_loader:
        X1_test = X1_test.to(device)
        X2_test = X2_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(X1_test, X2_test)
        prices_mean = torch.mean(Y_test)
        ss_tot += torch.sum((Y_test - prices_mean) ** 2)
        ss_res += torch.sum((Y_test - prediction) ** 2)
    
    accuracy = 1 - ss_res/ss_tot
    print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 84.00%
